### Useful links:
- [Dataset documentation](https://nijianmo.github.io/amazon/index.html)
- [Complete Metadata files](http://deepyeti.ucsd.edu/jianmo/amazon/index.html)
- [Pandas reference sheet](https://ds100.org/sp21/resources/assets/exams/sp20/sp20_checkpoint_reference_sheet.pdf)
- [Data-200 Google Doc](https://docs.google.com/document/d/19HWODy5kpWoUB7BEKEmKLbRnK8MC1fBmRat_WP7vfNc/edit)
- [Grad Project Guidelines](https://ds100.org/sp21/grad_proj/gradproject/)
- [Git repo](https://github.com/alexander-zw/data200-proj)

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import gzip
import urllib.request
import seaborn as sns
from urllib.request import urlopen

In [ ]:
url = "http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Software_5.json.gz"
filename = 'Softwares.json.gz'
if not os.path.exists(filename):
    urllib.request.urlretrieve(url,filename)


In [ ]:
### load the data

data = []
with gzip.open(filename) as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of reviews
print(len(data))

# first row of the list
print(data[0])

#### Convert to dataframe:

In [ ]:
reviews = pd.DataFrame.from_dict(data)
reviews.head()

In [ ]:
# Check score-wise values
reviews[(reviews['overall'] == 5)]

#### Column labels:
- reviewerID - ID of the reviewer, e.g. A2SUAM1J3GNN3B
- asin - ID of the product, e.g. 0000013714
- reviewerName - name of the reviewer
- vote - helpful votes of the review
- style - a disctionary of the product metadata, e.g., "Format" is "Hardcover"
- reviewText - text of the review
- overall - rating of the product
- summary - summary of the review
- unixReviewTime - time of the review (unix time)
- reviewTime - time of the review (raw)
- image - images that users post after they have received the product

#### Checking 5-core:
A 5-core dataset contains only those users with at least 5 reviews.

In [ ]:
reviews['reviewerID'].value_counts()

In [ ]:
reviews.query("reviewerID in ['A2AEZQ3DGBBLPR','A1PPD5TOR6VVYV','A3FGJDBSMCSG7G']")

#### Import metadata:

In [ ]:
url = "http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Software.json.gz"
filename = 'Meta_Softwares.json.gz'
if not os.path.exists(filename):
    urllib.request.urlretrieve(url,filename)

In [ ]:
### load the data

data = []
with gzip.open(filename) as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])

#### Convert to dataframe:

In [ ]:
metadata = pd.DataFrame.from_dict(data)
metadata.head()
metadata.shape

In [ ]:
metadata['asin'].value_counts().sort_values()

#### Merging the reviews and metadata on `asin`:

In [ ]:
df = reviews.merge(metadata,how="left",on = "asin")

In [ ]:
df.shape

In [ ]:
df.head()

#### Column labels:
- asin - ID of the product, e.g. 0000031852
- title - name of the product
- feature - bullet-point format features of the product
- description - description of the product
- price - price in US dollars (at time of crawl)
- image - url of the product image
- related - related products (also bought, also viewed, bought together, buy after viewing)
- salesRank - sales rank information
- brand - brand name
- categories - list of categories the product belongs to
- tech1 - the first technical detail table of the product
- tech2 - the second technical detail table of the product
- similar - similar product table

#### We can clean the data a little: 
- Change `overall` column name to `rating`
- `asin` to `productid`
- Extract `gift_amount` from `style`
- Extract `rank#` from `rank`
- Rempve rows with no information on the price either from `price` column or from `gift_amount` as both should be the same
- `price` missing? Change dataset?

### Data cleaning and filtering and EDA:

#### 1. Remove rows with unformatted title (i.e. some 'title' may still contain html style content)

In [ ]:
df3 = df.fillna('')
df4 = df3[df3.title.str.contains('getTime')] # unformatted rows
df5 = df3[~df3.title.str.contains('getTime')] # filter those unformatted rows
df = df5
print(len(df3))
print(len(df5))

#### 2. Check if there are products with missing price

In [ ]:
metadata[metadata['price']==''].shape

Indeed, products with missing price do exist. We now decide on whether we remove these products from the review dataset or we keep them and try coming up with a approximation for their price.
Let's check what proportion of the product dataset is missing a price:

In [ ]:
print("About ",metadata[metadata['price']==''].shape[0]/metadata.shape[0] * 100,"% products are missing a price value")

This proportion is very huge. Let's see if we can identify a pattern in the product metadata for missing prices.

In [ ]:
noprice = metadata[metadata['price'] =='']
withprice = metadata[metadata['price'] !='']
brands_no_price = noprice['brand'].value_counts(dropna=False).to_frame().reset_index()
brands_no_price.columns = ['brand','no_price']
brands_with_price = withprice['brand'].value_counts(dropna=False).to_frame().reset_index()
brands_with_price.columns = ['brand','with_price']

Checking if not having prices is a brand specific pattern or product specific. If this is not specific to a brand, we can use the average price for all the products of a brand to assign the missing price. Unfortuately, if there is a brand with no products priced, we will have to discard that data as the proportion of products with missing price data is already very big and making assumptions will skew the data.

`price_exist` is a dataframe with `brand` name,`total` # products, # products with price (`with_price`), # products without price(`no_price`) and a boolean feature which is `True` if the brand has both priced and unpriced products otherwies `False`.

In [ ]:
price_exist = metadata['brand'].value_counts(dropna=False).to_frame().reset_index()
price_exist.columns = ['brand','total']
price_exist.head()

price_exist = price_exist.merge(brands_with_price,how = 'left',on = 'brand').merge(brands_no_price,how = 'left',on = 'brand')

In [ ]:
price_exist.head()
price_exist.fillna(0)

It seems many brands have more products with no price in the dataset than otherwise. Let's see if the product itself is free on Amazon or if the dataset has missing data. Looking at products sold by Microsoft.

In [ ]:
metadata.query("brand == 'Microsoft'").head()

In [ ]:
# pd.set_option('display.max_columns', )  
print(metadata.query("brand== 'Microsoft'").loc[69,'description'],"\n")
print(metadata.query("brand== 'Microsoft'").loc[69,'title'])


After checking some of these products on Amazon.com, it appears that all these products offer outdated technologies. These products are "Currently Unavailable" on Amazon and have probably been so for a while since this dataset is about 3-4 years old.
Let's see what does the plot for total products to products that are currently unavailable looks like:

In [ ]:
plt.scatter(price_exist['total'],price_exist['no_price']/price_exist['total'],)

But this plot includes a lot of brands which only ever sold a few products and then shut down. Let's just look at the active brands. These are the brands which have atleast a few products with prices.

In [ ]:
filtered = price_exist['with_price']>10
x = price_exist.loc[filtered,'total']
y = price_exist.loc[filtered,'no_price']/price_exist.loc[filtered,'total']
plt.scatter(x,y)

Now let's just look at brands that sell more than a 100 products and are currently active.
We update our filter.

In [ ]:
filtered = (price_exist['with_price']>10) & (price_exist['total']>100)
x = price_exist.loc[filtered,'total']
y = price_exist.loc[filtered,'no_price']/price_exist.loc[filtered,'total']
plt.scatter(x,y)
plt.ylim(0,1);

It is interesting to see that most of these brands have more unavailable products than available products (i.e. proportion > 0.5). This clearly is an indicator of how quickly the inventory evolves especially in the tech/software field. The right extreme marker is Microsoft. And with a total of roughly 1200 products it is only actively selling about 175.

#### 3. Extracting the sentiment of the reviews

We will now proceed with extracting the sentiment from the reviews. We will then compare it with the rating and see what correlations we expect. One should expect a positive correlation. We will then compare sentiment against the prices

We shall be using VADER. The cell below loads the data containing all sentiments into a dataframe called `sent`

In [ ]:
print(''.join(open("vader_lexicon.txt").readlines()[:10]))
sent = pd.read_csv('vader_lexicon.txt',sep = '\t',index_col=0,header=None, usecols = [0,1], names = ['token',"polarity"])
sent.head()

Let's again look at the `reviews` dataframe

In [ ]:
reviews.head()

We are interested in the `summary` and `reviewText`

Before moving ahead we need to first remove all the punctuation. Below is a regex which will capture all the punctuations within text.

In [ ]:
punct_re = r'[^\w_\s]'

In [ ]:
def sanitize_texts(df):
    punct_re = r'[^\w_\s\n]'    
    df["clean_text"] = df['reviewText'].str.lower().str.replace(punct_re," ",regex = True)
    df["clean_summary"] = df['summary'].str.lower().str.replace(punct_re," ",regex = True)
    return df

reviews = sanitize_texts(reviews)
reviews["clean_text"].head()

In [ ]:
reviews["unixReviewTime"].value_counts()

It seems our review dataset doesn't have a primary key. Let's create one!!

In [ ]:
reviews["primary_key"] = reviews.index

In [ ]:
reviews["primary_key"].value_counts()

We ended up using the index as our primary key.

In [ ]:
def to_tidy_format(df):
    tidy = (
        df["clean_text"]
        .str.split()
        .explode()
        .to_frame()
        .rename(columns={"clean_text": "word"})
    )
    return tidy

tidyReviewText = to_tidy_format(reviews)
tidyReviewText.head()

### Adding in the Polarity Score

Now that we have this table in the tidy format, it becomes much easier to find the sentiment of each review: we can join the table with the lexicon table. 

In [ ]:
def add_polarity(df, tidy_df):
    df["polarity"] = (
        tidy_df
        .merge(sent, how='left', left_on='word', right_index=True)
        .reset_index()
        .loc[:, ['index', 'polarity']]
        .groupby('index')
        .sum()
        .fillna(0)
    )
    return df

reviews = add_polarity(reviews, tidyReviewText)
reviews.query("polarity<0")

With the polarity in place, let's see how polarity correlates with rating. We'll first do so without grouping by product and just focus on reviews.

In [ ]:
sns.boxplot(x="overall",y ="polarity",data = reviews)
plt.ylim(-100,100)

This is wierd. One would expect that the polarity of reviews would go up as the rating increases. Indeed, this is the case till about 4.0 but then the boxplot distribution shows a dip. Let's compare the correlation now for verified and non-verified users separately.

In [ ]:
sns.boxplot(x="overall",y ="polarity",data = reviews, hue = 'verified')
plt.ylim(-100,100)

This tells us that the dip in rating comes from unverified users. Let's also see some of the 5.0 rating reviews with polarity <0. It may be possible that VADER may not have some of these words.

In [ ]:
reviews.query("(polarity<10) & (overall == 5)").loc[:,['overall','reviewText','polarity']]

From the entries above it appears that these reviews are in general shorter in length. Which means that less polarity being added together. 

We will now create similar plots but for each product.

In [ ]:
product_reviews = reviews[['asin','overall','polarity']].groupby(by = 'asin').agg(np.mean)
product_reviews.head()

In [ ]:
product_reviews.plot.scatter('overall','polarity')

Now let's look at how price affects rating and polarity

In [ ]:
noprice_reviews

In [ ]:
noprice_reviews = df.loc[df['price']=='',:]
df.shape

Check for their brands:

In [ ]:
np.shape(noprice_reviews['brand'].unique()

Create a dataframe `product` with only the relevant columns: